# Práctica 1

In [1]:
#Librerias Necesarias
import pandas as pd
import numpy as np
import unicodedata
import re

In [2]:
#Funciones
def raname_columns(df,feats,prefix):
    feats_new=[prefix+x for x in feats]
    df=df.rename(columns=dict(zip(feats,feats_new)))
    return df

def completitud(df):
    comp=pd.DataFrame(df.isnull().sum())
    comp.reset_index(inplace=True)
    comp=comp.rename(columns={"index":"columna",0:"total"})
    comp["completitud"]=(1-comp["total"]/df.shape[0])*100
    comp=comp.sort_values(by="completitud",ascending=True)
    comp.reset_index(drop=True,inplace=True)
    return comp
#Funcion para limpiar caracteres especiales
def clean_text(text, pattern="[^a-zA-Z0-9 ]",replace=" "):
    cleaned_text = unicodedata.normalize('NFD', text).encode('ascii', 'ignore')
    cleaned_text = re.sub(pattern, replace, cleaned_text.decode("utf-8"), flags=re.UNICODE)
    cleaned_text = u' '.join(cleaned_text.strip().lower().split())
    return cleaned_text

In [3]:
#Datasets
df=pd.read_csv("spotify_practica.csv")
zips=pd.read_csv("zips_practica.csv")

/home/carlita/envs/diplomado/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<h2 style="color:green">(a) Realiza el etiquetado de las variables de acuerdo a su tipo de variable <h2/>

In [4]:
df["popularity"].value_counts()

0.0      27183
42.0      3202
43.0      3048
40.0      3002
44.0      2976
         ...  
93.0         3
97.0         1
99.0         1
96.0         1
100.0        1
Name: popularity, Length: 100, dtype: int64

In [5]:
#Continuas
c_feats=["duration_ms","loudness","tempo","speechiness","acousticness","danceability",
        "energy","instrumentalness","liveness","valence","zip Code"]
#Discretas
v_feats=["explicit","key","mode","popularity","genero"]
#Fechas
d_feats=["release_date"] 
#Texto
t_feats=["artists","name"] 

df=raname_columns(df,c_feats,"c_")
df=raname_columns(df,v_feats,"v_")
df=raname_columns(df,t_feats,"t_")
df=raname_columns(df,d_feats,"d_")


In [6]:
df.head()

,c_acousticness,t_artists,c_danceability,c_duration_ms,c_energy,v_explicit,id,c_instrumentalness,v_key,c_liveness,c_loudness,v_mode,t_name,v_popularity,d_release_date,c_speechiness,c_tempo,c_valence,c_zip Code,v_genero
0,0.764,['Shel Silverstein'],0.669,210387,0.1720,NaN,1r3N7MoEGPHJyCjBlWCsx3,0.000000,2.0,0.1280,-13.075,1,The Unicorn,12.0,1962-10-13 00:00:00,0.3270,97.079,0.333,38ee029,NaN
1,0.917,['Mose Allison'],0.693,149547,0.2410,NaN,2RL8EkOOu0QhAikhUUvIRi,0.000014,NaN,0.0824,-14.605,0,If You Live,19.0,1963-04-14 00:00:00,0.0350,120.890,0.715,9p2304,NaN
2,0.926,['水柳仙'],0.183,207173,0.0854,0.0,3kKKEghp2EMh8992pWFylr,0.704000,7.0,0.1120,-26.598,1,鴛鴦夢,23.0,1963-11-14 00:00:00,0.0392,101.482,0.131,2166ee6,NaN
3,0.976,['Dean Martin'],0.472,182653,0.0314,NaN,0j3ideT0tRuIyRhOOUQjYa,0.000015,7.0,0.1500,-18.006,1,Fools Rush In,21.0,1964-06-07 00:00:00,0.0364,135.555,0.194,67t06,NaN
4,0.892,['Doris Day'],0.243,171400,0.3360,NaN,0od9zbD6Bcc8b7dCMRFWBZ,0.000049,2.0,0.1310,-9.812,0,The Christmas Waltz,22.0,1964-08-07 00:00:00,0.0282,89.158,0.238,804p53,NaN


<h2 style="color:green">(b) Eliminar duplicados <h2/>

In [7]:
#Número de Duplicados 
df.duplicated().sum()

364

In [8]:
#Eliminar duplicados
df.drop_duplicates(inplace=True)

In [9]:
#Duplicados por id
df.duplicated(subset=['id']).sum()

76

In [10]:
#Eliminar uplicados por id
df.drop_duplicates(subset=['id'],inplace=True)

In [11]:
df.reset_index(drop=True,inplace=True)

<h2 style="color:green">(c) Completitud<h2/>

In [12]:
completitud(df)

,columna,total,completitud
0,v_explicit,62883,62.990189
1,v_key,35746,78.961679
2,v_popularity,3500,97.940074
3,v_genero,1698,99.000642
4,c_valence,0,100.000000
5,c_tempo,0,100.000000
6,c_speechiness,0,100.000000
7,d_release_date,0,100.000000
8,t_name,0,100.000000
9,v_mode,0,100.000000



<h2 style="color:green">(d) Elimina las variables que tengan el 20 % o mas de valores ausentes <h2/>


In [13]:
#Columnas que tengan una completitud inferior al 80%
df.drop(columns=["v_explicit","v_key"],inplace=True)
df.shape

(169909, 18)

<h2 style="color:green">(e) ¿Cuantos de los registros en la variable ”zip Code” son valores no validos? , es decir, contienen letras<h2/>

In [14]:
#Registros que contienen letras
df[~df["c_zip Code"].map(lambda x:str(x).isdigit())].shape[0]

4341

In [15]:
#Registros que no contienen letras
df[df["c_zip Code"].map(lambda x:str(x).isdigit())]["c_zip Code"].shape[0]

165568

<h2 style="color:green">(f) Elimina los registros que no tengas un ”zip Code” valido, es decir contenga letras en los valores<h2/>

In [16]:
#Elimina los registros no validos
df=df[df["c_zip Code"].map(lambda x:str(x).isdigit())].reset_index(drop=True)

In [17]:
df.shape

(165568, 18)

<h2 style="color:green">(g) ¿Cuantos de los registros en la variable ”genero” son valores no validos? , es decir, contienen letras<h2/>

In [18]:
aux1=df[df["v_genero"].isnull()]
aux2=df[df["v_genero"].notnull()]

In [19]:
#Registros que contienen letras
aux2[aux2["v_genero"].map(lambda x:sum([x.isalpha() for x in str(x).replace(".","")])>0)]

,c_acousticness,t_artists,c_danceability,c_duration_ms,c_energy,id,c_instrumentalness,c_liveness,c_loudness,v_mode,t_name,v_popularity,d_release_date,c_speechiness,c_tempo,c_valence,c_zip Code,v_genero
7,0.000186,['Green Day'],0.494,128200,0.9320,2uARrCX71Zl7JspUN2KuPZ,0.00004,0.0248,-4.606,0,Bab's Uvula Who?,NaN,1995-01-18 00:00:00,0.0542,96.215,0.9720,74023,Часть 2.3 & Часть 3.1
41,0.887000,"['Max Bruch', 'Yehudi Menuhin', 'Charles Münch']",0.225,466440,0.3070,31VWjwLGWFmEV99kR2fOTF,0.55100,0.1180,-15.181,1,"Violin Concerto No. 1 in G Minor, Op. 26: II. ...",0.0,1945-07-15 00:00:00,0.0406,102.523,0.1480,54889,3. Sure
1644,0.994000,"['Robert Schumann', 'Vladimir Horowitz']",0.379,282133,0.0135,6KuQTIu1KoTTkLXKrwlLPV,0.90100,0.0763,-28.454,1,"Fantasiestücke, Op. 111: Più tosto lento",0.0,1928-11-13 00:00:00,0.0462,83.972,0.0767,81655,Bebamos Juntos
1645,0.995000,['Francisco Canaro'],0.781,180760,0.1300,6M94FkXd15sOAOQYRnWPN8,0.88700,0.1110,-14.734,0,Bebamos Juntos - Instrumental (Remasterizado),0.0,1928-04-13 00:00:00,0.0926,108.003,0.7200,75605,Cristian
1646,0.995000,"['Felix Mendelssohn', 'Vladimir Horowitz']",0.424,352600,0.1200,6NxAf7M8DNHOBTmEd3JSO5,0.91100,0.0915,-19.242,0,Scherzo a capriccio: Presto,NaN,1928-04-10 00:00:00,0.0593,63.521,0.2660,16370,2Woitschach
1647,0.988000,['Carl Woitschach'],0.555,153967,0.4210,6OJjveoYwJdIt76y0Pxpxw,0.83600,0.1050,-9.878,1,Per aspera ad astra,0.0,1928-10-03 00:00:00,0.0474,123.310,0.8570,61274,Fantasiestücke
1648,0.990000,"['Samuel Barber', 'Vladimir Horowitz']",0.359,338333,0.0435,6QgdUySTRGVkNo3KwbHpK3,0.89900,0.1090,-20.858,1,"Piano Sonata in E-Flat Minor, Op. 26: III. Ada...",0.0,1928-03-16 00:00:00,0.0424,96.645,0.0420,68875,Polonaise
1649,0.975000,"['Frédéric Chopin', 'Vladimir Horowitz']",0.277,245440,0.0900,6VUm7Dg5sufmG01IYcoJE3,0.94900,0.1250,-26.188,1,"Andante spianato in E-Flat Major, Op. 22",0.0,1928-03-01 00:00:00,0.0316,105.031,0.1680,99009,6N6tiFZ9vLTSOIxkj8qKrd
1650,0.987000,"['Samuel Barber', 'Vladimir Horowitz']",0.242,115627,0.1490,6v13FCz4z385EbNyPaXYCU,0.87700,0.1340,-26.742,1,"Piano Sonata in E-Flat Minor, Op. 26: II. Alle...",0.0,1928-09-24 00:00:00,0.0369,88.188,0.2790,55066,6L63VW0PibdM1HDSBoqnoM
1651,0.990000,"['Frédéric Chopin', 'Vladimir Horowitz']",0.272,473227,0.2810,6wt8PKNBSqNFqdOxgexvKu,0.86700,0.0870,-16.228,0,"Piano Sonata No. 2, Op. 35: I. Grave - Doppio ...",0.0,1928-08-19 00:00:00,0.0365,99.270,0.0775,7040,Хемингуэй


In [20]:
#Número de registros que contienen letras
aux2[aux2["v_genero"].map(lambda x:sum([x.isalpha() for x in str(x).replace(".","")])>0)].shape[0]

10

<h2 style="color:green">(h) Elimina los registros que no tengas un ”genero” valido, es decir contenga letras en los valores<h2/>


In [21]:
#Elimina los registros no validos
print(aux2.shape)
aux2=aux2[~aux2["v_genero"].map(lambda x:sum([y.isalpha() for y in str(x)])>0)].reset_index(drop=True)
print(aux2.shape)

(163926, 18)
(163916, 18)


In [22]:
print(df.shape)
df=pd.concat([aux2,aux1],ignore_index=True)
print(df.shape)

(165568, 18)
(165558, 18)


<h2 style="color:green">(i) Haz una limpieza de la variable ”name” , elimina caracteres especiales y todo debe estar en minusculas<h2/>

In [23]:
#Limpieza variable name
df["t_name"]=df["t_name"].map(lambda x:clean_text(x.lower(),replace=" "))
df["t_name"].head()

0    sunless saturday
1        read my lips
2         rain forest
3     praise of death
4       swingin party
Name: t_name, dtype: object

<h2 style="color:green">
(j) De la variable ”artist” selecciona solo al primer artista que aparezca en la lista, ademas de eliminar caracteres especiales<h2/>



In [24]:
#Utilizamos "eval" para convertir los strings "[1]" a una lista en python y poder hacer seleccion de elementos
df["t_artists"].map(lambda x:eval(x)[0]) 

0                    Fishbone
1              Michael Franks
2                  Paul Speer
3                      Slayer
4            The Replacements
                 ...         
165553              Pop Smoke
165554           Lil Uzi Vert
165555     Alexander Scriabin
165556    Seweryn Goszczyński
165557            The Merlons
Name: t_artists, Length: 165558, dtype: object

In [25]:
df["t_artists"]=df["t_artists"].map(lambda x:eval(x)[0]) 

In [26]:
#Limpiamos la variable
df["t_artists"]=df["t_artists"].map(lambda x:clean_text(x,replace=" "))

In [27]:
df.head()

,c_acousticness,t_artists,c_danceability,c_duration_ms,c_energy,id,c_instrumentalness,c_liveness,c_loudness,v_mode,t_name,v_popularity,d_release_date,c_speechiness,c_tempo,c_valence,c_zip Code,v_genero
0,0.000069,fishbone,0.451,258440,0.970,4dtBF3w1v4NLzkO6ePwvue,0.10300,0.1360,-4.622,1,sunless saturday,31.0,1985-03-10 00:00:00,0.1360,144.071,0.3960,65774,47.0
1,0.068200,michael franks,0.861,222107,0.726,68LFkY4TMwgKqfOtnNEZ1j,0.00013,0.0922,-12.759,1,read my lips,35.0,1985-11-24 00:00:00,0.0743,118.005,0.8980,92586,20.0
2,0.936000,paul speer,0.605,258000,0.229,6FZfxmeVhsdqkD8WJfP83n,0.86000,0.1000,-16.300,1,rain forest,32.0,1985-02-27 00:00:00,0.0340,127.295,0.0634,918,50.0
3,0.001650,slayer,0.267,321040,0.957,6hIroseB9TaxU9a5lhO4sD,0.40800,0.0897,-7.754,0,praise of death,33.0,1985-07-05 00:00:00,0.0812,104.683,0.1410,59247,50.0
4,0.028900,the replacements,0.644,231267,0.629,789VjU3oh8sxOfBN4fupqH,0.00000,0.0662,-11.646,0,swingin party,27.0,1985-06-09 00:00:00,0.0473,117.489,0.7480,2919,20.0


<h2 style="color:green">(k) Normaliza la variable ”genero” de tal forma que obtengas solo * categorıas<h2/>

In [28]:
df["v_genero"].value_counts()

47.0    27167
20.0    24855
50.0    20144
12.0    18301
16.0    18054
21.0    17829
47.0     5812
20.0     5398
50.0     4247
21.0     3899
16.0     3843
12.0     3774
30.0     2653
49.0     2627
48.0     2565
30.0      579
49.0      566
48.0      551
3.0       448
2.0       285
1.0       147
3.0        84
2.0        65
1.0        23
Name: v_genero, dtype: int64

In [29]:
#Valores de genero a tipo flotante
df["v_genero"]=df["v_genero"].map(float)

In [30]:
#Presencia de las categorias
df["v_genero"].value_counts(1)

47.0    0.201195
20.0    0.184564
50.0    0.148802
12.0    0.134673
16.0    0.133587
21.0    0.132556
30.0    0.019717
49.0    0.019479
48.0    0.019010
3.0     0.003246
2.0     0.002135
1.0     0.001037
Name: v_genero, dtype: float64

In [31]:
#Las ultimas tres valores categorias las agrupamos en una categoria "otros", de tal forma que represente el porcenntaje mas pequeño
dictio=dict(zip(list(df["v_genero"].value_counts(1).index.tolist()[-3:]),["otros"]*3))

In [32]:
#Reemplazamos la categoria de "otros"
df["v_genero"]=df["v_genero"].replace(dictio)

In [33]:
#Categorias obtenidas
df["v_genero"].value_counts()

47.0     32979
20.0     30253
50.0     24391
12.0     22075
16.0     21897
21.0     21728
30.0      3232
49.0      3193
48.0      3116
otros     1052
Name: v_genero, dtype: int64

<h2 style="color:green">(l) Añade las siguientes columnas a tu tabla base: zip, lat, lng, city, state name utilizando la tabla zips practica<h2/>

In [34]:
#Mismo tipo de dato que zip_code
df["c_zip Code"]=df["c_zip Code"].astype(int)
df["c_zip Code"].dtype

dtype('int64')

In [35]:
#Seleccionamos solo las variables necesarias
zips=zips[['zip','lat','lng','city','state_name']]
zips

,zip,lat,lng,city,state_name
0,601,18.18004,-66.75218,Adjunta28s,Puerto 345Rico
1,602,18.36073,-67.17517,A543guada,P733uerto Rico
2,603,18.45439,-67.12202,Ag188uadilla,Pue888rto Rico
3,606,18.16724,-66.93828,Ma297ricao,Puerto Ric683o
4,610,18.29032,-67.12243,Anasc449o,P753uerto Rico
...,...,...,...,...,...
33094,99923,55.97796,-130.03671,H481yder,A540laska
33095,99925,55.55767,-132.97627,442Klawock,Alask533a
33096,99926,55.12617,-131.48928,M165etlakatla,797Alaska
33097,99927,56.25100,-133.37571,Po905int Baker,Alask349a


In [36]:
#Merge de los zips a la tabla base
df=pd.merge(df,zips,left_on='c_zip Code',right_on='zip',how='left')

In [37]:
df.head()

,c_acousticness,t_artists,c_danceability,c_duration_ms,c_energy,id,c_instrumentalness,c_liveness,c_loudness,v_mode,...,c_speechiness,c_tempo,c_valence,c_zip Code,v_genero,zip,lat,lng,city,state_name
0,0.000069,fishbone,0.451,258440,0.970,4dtBF3w1v4NLzkO6ePwvue,0.10300,0.1360,-4.622,1,...,0.1360,144.071,0.3960,65774,47,65774,37.92914,-93.49222,We801aubleau,Misso551uri
1,0.068200,michael franks,0.861,222107,0.726,68LFkY4TMwgKqfOtnNEZ1j,0.00013,0.0922,-12.759,1,...,0.0743,118.005,0.8980,92586,20,92586,33.70901,-117.19889,Menife875e,Cali972fornia
2,0.936000,paul speer,0.605,258000,0.229,6FZfxmeVhsdqkD8WJfP83n,0.86000,0.1000,-16.300,1,...,0.0340,127.295,0.0634,918,50,918,18.42138,-66.06588,San706 Juan,Puerto Ri109co
3,0.001650,slayer,0.267,321040,0.957,6hIroseB9TaxU9a5lhO4sD,0.40800,0.0897,-7.754,0,...,0.0812,104.683,0.1410,59247,50,59247,48.51172,-104.53507,Medic882ine Lake,Mo655ntana
4,0.028900,the replacements,0.644,231267,0.629,789VjU3oh8sxOfBN4fupqH,0.00000,0.0662,-11.646,0,...,0.0473,117.489,0.7480,2919,20,2919,41.82737,-71.51998,Joh127nston,Rhode Is658land


In [38]:
#Añadimos prefijo a las nuevas variables
df=raname_columns(df,["lat","lng"],"c_")
df=raname_columns(df,["city","state_name"],"t_")

In [39]:
df

,c_acousticness,t_artists,c_danceability,c_duration_ms,c_energy,id,c_instrumentalness,c_liveness,c_loudness,v_mode,...,c_speechiness,c_tempo,c_valence,c_zip Code,v_genero,zip,c_lat,c_lng,t_city,t_state_name
0,0.000069,fishbone,0.451,258440,0.970,4dtBF3w1v4NLzkO6ePwvue,0.10300,0.1360,-4.622,1,...,0.1360,144.071,0.3960,65774,47,65774,37.92914,-93.49222,We801aubleau,Misso551uri
1,0.068200,michael franks,0.861,222107,0.726,68LFkY4TMwgKqfOtnNEZ1j,0.00013,0.0922,-12.759,1,...,0.0743,118.005,0.8980,92586,20,92586,33.70901,-117.19889,Menife875e,Cali972fornia
2,0.936000,paul speer,0.605,258000,0.229,6FZfxmeVhsdqkD8WJfP83n,0.86000,0.1000,-16.300,1,...,0.0340,127.295,0.0634,918,50,918,18.42138,-66.06588,San706 Juan,Puerto Ri109co
3,0.001650,slayer,0.267,321040,0.957,6hIroseB9TaxU9a5lhO4sD,0.40800,0.0897,-7.754,0,...,0.0812,104.683,0.1410,59247,50,59247,48.51172,-104.53507,Medic882ine Lake,Mo655ntana
4,0.028900,the replacements,0.644,231267,0.629,789VjU3oh8sxOfBN4fupqH,0.00000,0.0662,-11.646,0,...,0.0473,117.489,0.7480,2919,20,2919,41.82737,-71.51998,Joh127nston,Rhode Is658land
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165553,0.143000,pop smoke,0.836,194680,0.627,1dnsojUNfvhiztcITLzruj,0.00000,0.0657,-4.377,0,...,0.1970,142.990,0.6520,41004,NaN,41004,38.65389,-84.10655,Brooksv160ille,Ke50ntucky
165554,0.023600,lil uzi vert,0.775,234627,0.720,1Xd0PWZV3LCEGzc7fkjTeu,0.00000,0.1140,-5.353,0,...,0.1930,155.086,0.4900,82001,NaN,82001,41.09146,-104.93445,Che214yenne,Wyomin654g
165555,0.996000,alexander scriabin,0.319,326067,0.155,6T0mZB7p3qzOifdXZ3Su9P,0.91700,0.1260,-18.728,1,...,0.0360,66.947,0.0488,12935,NaN,12935,44.83473,-73.78753,Ellenburg 249Depot,Ne100w York
165556,0.732000,seweryn goszczynski,0.720,100700,0.286,6qxoAGlWUrVYbvq3x6S08C,0.00000,0.2920,-15.861,1,...,0.9390,114.059,0.6950,7058,NaN,7058,40.86728,-74.34275,Pine Br352ook,677New Jersey


<h2 style="color:green"> (m) Convierte las variables ”lat” y ”lng” en tipo flotante y valida que la informacion sea consistente.<h2/>

In [40]:
#Vriable lat a tipo flotante
df["c_lat"]=df["c_lat"].astype(float)

In [41]:
#Validamos la informacion (latitud solo toma valores mayores o igual a -90 y menores a 90)

df[(df["c_lat"]<-90)|(df["c_lat"]>90)]

,c_acousticness,t_artists,c_danceability,c_duration_ms,c_energy,id,c_instrumentalness,c_liveness,c_loudness,v_mode,...,c_speechiness,c_tempo,c_valence,c_zip Code,v_genero,zip,c_lat,c_lng,t_city,t_state_name


In [42]:
#Variable longitud a tipo flotante
df["c_lng"]=df["c_lng"].astype(float)

In [43]:

df["c_lng"].describe()

count    165558.000000
mean        -91.209638
std          16.185693
min        -331.846760
25%         -97.279790
50%         -88.259090
75%         -80.222950
max         -65.290170
Name: c_lng, dtype: float64

In [44]:
#Observamos valores menores que -180
df[(df["c_lng"]<-180)|(df["c_lng"]>180)]

,c_acousticness,t_artists,c_danceability,c_duration_ms,c_energy,id,c_instrumentalness,c_liveness,c_loudness,v_mode,...,c_speechiness,c_tempo,c_valence,c_zip Code,v_genero,zip,c_lat,c_lng,t_city,t_state_name
1116,0.25500,men at work,0.708,271707,0.605,5xNmWMhsZncmM9zziHww42,0.000022,0.0838,-4.710,1,...,0.0287,106.297,0.9360,14714,16,14714,42.28999,-258.23324,Bla805ck Creek,New 276York
1318,0.52100,fred neil,0.671,164600,0.299,7GrpcaiW2YyqUH3n1F6Q6s,0.000288,0.1500,-13.215,1,...,0.0315,120.112,0.5740,61075,20,61075,42.46444,-270.25896,485Scales Mound,Ill85inois
2599,0.44200,alexandre desplat,0.321,144200,0.241,775OxWq9wfVzmVDS0n3zff,0.888000,0.1050,-20.495,1,...,0.0373,128.980,0.0795,61243,47,61243,41.61876,-269.68295,Deer Gro891ve,Illino520is
2673,0.05910,mumford sons,0.454,290813,0.415,0cBDCol7TFmvpT4A4YpxV3,0.003610,0.0831,-9.554,1,...,0.0359,129.621,0.1360,38069,50,38069,35.46389,-269.36435,56Stanton,T65ennessee
2732,0.12000,luke benward,0.703,203320,0.691,686IBfvmRBy1eFjYLPIJFr,0.000000,0.0830,-7.294,1,...,0.0315,121.016,0.9060,61243,20,61243,41.61876,-269.68295,Deer Gro891ve,Illino520is
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160882,0.84300,ella mae morse,0.761,182227,0.182,5A0SH2AUQa0iyaVkTs1wDu,0.000022,0.0761,-12.773,0,...,0.1030,128.326,0.5860,48173,16,48173,42.07512,-263.21466,R330ockwood,Michig181an
160929,0.88400,the fleetwoods,0.588,128867,0.158,6uQlc0WC4jGIJhIKCIfHJo,0.000648,0.1110,-17.737,1,...,0.0341,105.659,0.4720,48173,16,48173,42.07512,-263.21466,R330ockwood,Michig181an
161434,0.00239,dave simon,0.458,286567,0.955,0IM4J8XbrTKxe6xMiHYEFD,0.000002,0.1350,-4.335,0,...,0.0413,157.998,0.7070,61243,16,61243,41.61876,-269.68295,Deer Gro891ve,Illino520is
162205,0.07170,mary j blige,0.726,246333,0.700,6Y3WvyUG9iE5bQYg38SPtQ,0.000000,0.2620,-5.881,0,...,0.0998,146.559,0.6730,99756,12,99756,64.58632,-331.84676,Manley Hot Sprin273gs,Alask655a


In [45]:
#Número de registros no validos en longitud
df[(df["c_lng"]<-180)|(df["c_lng"]>180)].shape[0]

173

In [46]:
df.shape[0]

165558

In [47]:
#Eliminamos los registro no validos
df=df[~(df["c_lng"]<-180)|(df["c_lng"]>180)].reset_index(drop=True)

In [48]:
df.shape

(165385, 23)

<h2 style="color:green">(n) De la variable ”city” y ”state” elimina los dıgitos que se encuentran dentro de las cadenas de texto<h2/>

In [49]:
#Utilizamos clean_text y modificamos el pattern para que solo seleccione letras y los números los modifique por ""
df["t_city"].map(lambda x:clean_text(x, pattern="[^a-zA-Z ]",replace=""))

0               weaubleau
1                 menifee
2                san juan
3           medicine lake
4                johnston
               ...       
165380        brooksville
165381           cheyenne
165382    ellenburg depot
165383         pine brook
165384             verona
Name: t_city, Length: 165385, dtype: object

In [50]:
df["t_city"]=df["t_city"].map(lambda x:clean_text(x, pattern="[^a-zA-Z ]",replace=""))

In [51]:
df["t_state_name"].map(lambda x:clean_text(x, pattern="[^a-zA-Z ]",replace=""))

0             missouri
1           california
2          puerto rico
3              montana
4         rhode island
              ...     
165380        kentucky
165381         wyoming
165382        new york
165383      new jersey
165384     mississippi
Name: t_state_name, Length: 165385, dtype: object

In [52]:
df["t_state_name"]=df["t_state_name"].map(lambda x:clean_text(x, pattern="[^a-zA-Z ]",replace=""))

<h2 style="color:green">(o) Crea una nueva variable llamada ”state” que este conformada por ”city”,”state name”<h2/>

In [53]:
df["t_state"]=df["t_city"]+ ", " + df["t_state_name"]

df.head()

,c_acousticness,t_artists,c_danceability,c_duration_ms,c_energy,id,c_instrumentalness,c_liveness,c_loudness,v_mode,...,c_tempo,c_valence,c_zip Code,v_genero,zip,c_lat,c_lng,t_city,t_state_name,t_state
0,0.000069,fishbone,0.451,258440,0.970,4dtBF3w1v4NLzkO6ePwvue,0.10300,0.1360,-4.622,1,...,144.071,0.3960,65774,47,65774,37.92914,-93.49222,weaubleau,missouri,"weaubleau, missouri"
1,0.068200,michael franks,0.861,222107,0.726,68LFkY4TMwgKqfOtnNEZ1j,0.00013,0.0922,-12.759,1,...,118.005,0.8980,92586,20,92586,33.70901,-117.19889,menifee,california,"menifee, california"
2,0.936000,paul speer,0.605,258000,0.229,6FZfxmeVhsdqkD8WJfP83n,0.86000,0.1000,-16.300,1,...,127.295,0.0634,918,50,918,18.42138,-66.06588,san juan,puerto rico,"san juan, puerto rico"
3,0.001650,slayer,0.267,321040,0.957,6hIroseB9TaxU9a5lhO4sD,0.40800,0.0897,-7.754,0,...,104.683,0.1410,59247,50,59247,48.51172,-104.53507,medicine lake,montana,"medicine lake, montana"
4,0.028900,the replacements,0.644,231267,0.629,789VjU3oh8sxOfBN4fupqH,0.00000,0.0662,-11.646,0,...,117.489,0.7480,2919,20,2919,41.82737,-71.51998,johnston,rhode island,"johnston, rhode island"


<h2 style="color:green">(p) Los valores de la nueva variable ”state” modificalos de cierta forma que todas sean minusculas y sin acentos<h2/>

In [54]:
#Ya se eliminaron acentos en las variables city y state_name y solo verificamos que sean minusculas
df["t_state"]=df["t_state"].map(lambda x:x.lower())
df["t_state"].head()

0       weaubleau, missouri
1       menifee, california
2     san juan, puerto rico
3    medicine lake, montana
4    johnston, rhode island
Name: t_state, dtype: object

<h2 style="color:green">(q) Convierte los valores en la variable ”release date” a tipo datetime , ademas haz el conteo de las que no tienen la estructura necesaria para ser convertida en datetime y elimina esos registros<h2/>

In [55]:
#Convertimos a datetime
df["d_release_date"]=pd.to_datetime(df["d_release_date"])

<h2 style="color:orange">2.(a) Obten un DataFrame donde se muestre el conteo de registros por genero<h2/>

In [56]:
conteo_genero=df[["v_genero","id"]].groupby("v_genero").count().reset_index().rename({"id":"Conteo"})

In [57]:
conteo_genero

,v_genero,id
0,12,22051
1,16,21876
2,20,30224
3,21,21709
4,30,3225
5,47,32944
6,48,3113
7,49,3189
8,50,24361
9,otros,1051


<h2 style="color:orange">2.(b) Genera una nueva variable que se llame ”duration minutos” que sea el valor de la variable ”duration ms” en minutos<h2/>

In [58]:
#Definimos una funcion para convertir milisegundos a minutos
def minu(x):
    ms = int(x)
    s=(ms/1000)%60
    s = int(s)
    minutes=(ms/(1000*60))%60
    return minutes

In [59]:
#Dividimos los milisegundos por 60000 y lo convertimos a entero
df["c_duration_ms"].map(lambda x: int(x/60000))

0         4
1         3
2         4
3         5
4         3
         ..
165380    3
165381    3
165382    5
165383    1
165384    2
Name: c_duration_ms, Length: 165385, dtype: int64

In [60]:
df["c_duration_min"]=df["c_duration_ms"].map(lambda x: int(x/60000))

<h2 style="color:orange">2.(c) ¿Cuales son las 10 canciones mas populares?<h2/>

In [61]:
#Agrupamos por nombre de la cancion y buscamos el maximo de popularidad
df[['v_popularity','t_name']].groupby("t_name").agg("max").sort_values('v_popularity', ascending=False).head(10)

,v_popularity
t_name,
blinding lights,100.0
rockstar feat roddy ricch,99.0
death bed coffee for your head feat beabadoobee,97.0
the scotts,96.0
the box,95.0
toosie slide,95.0
supalonely,95.0
ily i love you baby feat emilee,95.0
dance monkey,94.0


<h2 style="color:orange">2.(d) ¿Cual es el promedio de duracion en minutos y milisegundos?<h2/>

In [62]:
print(f"El promedio de duracion en minutos es : {round(df['c_duration_min'].mean(),2)}")


El promedio de duracion en minutos es : 3.36


In [63]:

print(f"El promedio de duracion en milisegundos es : {round(df['c_duration_ms'].mean(),2)}")

El promedio de duracion en milisegundos es : 231413.97


<h2 style="color:orange">2.(e) ¿Cual es el promedio y conteo de ”energy” por ”genero”?<h2/>

In [64]:
df[["c_energy","v_genero"]].groupby("v_genero").agg(["mean","count"])

c_energy       
              mean  count
v_genero                 
12.0      0.493282  22051
16.0      0.488466  21876
20.0      0.488744  30224
21.0      0.489854  21709
30.0      0.488177   3225
47.0      0.491386  32944
48.0      0.493182   3113
49.0      0.489869   3189
50.0      0.492076  24361
otros     0.480729   1051

<h2 style="color:orange">2.(f) ¿Que cancion tiene el ”loudness” mas bajo y cual es la cancion que tiene el ”loudness” mas alto?<h2/>

In [65]:
#Loudness mas alto - Cancion
df[['c_loudness','t_name']].groupby("t_name").agg("max").sort_values('c_loudness', ascending=False).head(1)

,c_loudness
t_name,
sold,3.855


In [66]:
#Cancion con el loudness mas bajo
df[df["c_loudness"]==df["c_loudness"].min()]["t_name"]

37004                            hava nagilah
43513                             pause track
55878                            silent track
56760                            silent track
59349                       my yiddishe momme
93434                        pause track live
102717                       pause track live
131150    staggerlee has his day at the beach
139704                            pause track
Name: t_name, dtype: object

<h2 style="color:orange">2.(g) Obtenga los 10 percentiles de las variables continuas<h2/>

In [67]:
#Variables continuas
df.filter(like="c_").describe(percentiles=np.arange(0.1,1.1,.1))

,c_acousticness,c_danceability,c_duration_ms,c_energy,c_instrumentalness,c_liveness,c_loudness,c_speechiness,c_tempo,c_valence,c_zip Code,c_lat,c_lng,c_duration_min
count,165385.000000,165385.000000,1.653850e+05,165385.000000,165385.000000,165385.000000,165385.000000,165385.000000,165385.000000,165385.000000,165385.000000,165385.000000,165385.000000,165385.000000
mean,0.489770,0.539255,2.314140e+05,0.490662,0.160732,0.206409,-11.322550,0.094149,117.014726,0.532137,49678.013526,38.844631,-91.021895,3.358201
std,0.376817,0.175448,1.208784e+05,0.267560,0.308482,0.176539,5.666758,0.149633,30.736135,0.262245,27564.699930,5.336454,15.103526,2.035729
min,0.000000,0.000000,5.108000e+03,0.000000,0.000000,0.000000,-60.000000,0.000000,0.000000,0.000000,601.000000,17.966120,-176.631390,0.000000
10%,0.008710,0.299000,1.376530e+05,0.131000,0.000000,0.072900,-18.869000,0.030000,79.633000,0.159000,12494.000000,32.083210,-115.541160,2.000000
20%,0.051900,0.385000,1.625810e+05,0.222000,0.000000,0.091100,-15.565000,0.033400,89.268000,0.270800,21624.000000,34.309852,-99.660920,2.000000
30%,0.144000,0.448000,1.793046e+05,0.307000,0.000002,0.105000,-13.452000,0.036600,97.398200,0.367000,30669.000000,36.430390,-95.593110,2.000000
40%,0.289000,0.501000,1.936400e+05,0.396000,0.000020,0.116000,-11.855000,0.040400,106.043000,0.460000,40947.800000,38.155640,-91.911400,3.000000
50%,0.483000,0.549000,2.088750e+05,0.483000,0.000192,0.135000,-10.413000,0.045100,114.851000,0.544000,49807.000000,39.495250,-88.243500,3.000000
60%,0.680000,0.596000,2.275600e+05,0.572000,0.001940,0.167000,-9.066000,0.052200,122.720400,0.626000,58482.000000,40.602090,-84.602652,3.000000
